In [ ]:
# 데이터 입력
from google.colab import files
uploaded = files.upload()
my_data = 'sonar.csv'

!pip install -q tensorflow-gpu==1.15.0
%tensorflow_version 1.x
import tensorflow as tf
import pandas as pd
import numpy

from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.compat.v1.set_random_seed(3)

# 데이터 적용
df = pd.read_csv(my_data, header=None)

dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]

e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

Saving sonar.csv to sonar.csv
     |████████████████████████████████| 411.5MB 39kB/s 
     |████████████████████████████████| 512kB 45.2MB/s 
     |████████████████████████████████| 3.8MB 43.3MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
TensorFlow 1.x selected.


Using TensorFlow backend.


데이터를 원하는 숫자만큼 쪼개 각각 학습셋과 테스트셋으로 사용하게 만드는 함수는 sklearn의 StratifiedKFold() 함수

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 10개의 파일로 쪼갬
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

딥러닝 혹은 머신러닝 작업을 할 때 늘 어려운 문제 중 하나는 알고리즘을 충분히 테스트하였어도 데이터가 충분치 않으면 좋은 결과를 내기 어렵다는 것입니다. 이러한 단점을 보완하고자 만든 방법이 바로 k겹 교차 검증(k-fold cross validation)

**k겹 교차 검증:** 데이터셋을 여러 개로 나누어 하나씩 테스트셋으로 사용하고 나머지를 모두 합해서 학습셋으로 사용하는 방법. 이렇게 하면 데이터의 100%를 테스트셋으로 사용할 수 있습니다.

In [ ]:
# 빈 accuracy 배열
accuracy = []

# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

# 결과 출력
print("\n %.f fold accuracy:" % n_fold, accuracy)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/100
187/187 [==============================] - 0s 2ms/step - loss: 0.2469 - accuracy: 0.5348
Epoch 2/100
187/187 [==============================] - 0s 226us/step - loss: 0.2286 - accuracy: 0.7380
Epoch 3/100
187/187 [==============================] - 0s 232us/step - loss: 0.2172 - accuracy: 0.6738
Epoch 4/100
187/187 [==============================] - 0s 257us/step - loss: 0.1973 - accuracy: 0.7219
Epoch 5/100
187/187 [==============================] - 0s 264us/step - loss: 0.1882 - accuracy: 0.7701
Epoch 6/100
187/187 [==============================] - 0s 277us/step - loss: 0.1746 - accuracy: 0.7968
Epoch 7/100
187/187 [==============================] - 0s 258us/step - loss: 0.1661 - accuracy: 0.8289
Epoch 8/100
187/187 [==============================] - 0s 274us/step - loss: 0.1583 - accuracy: 0.7914
Epoch 9/100
187/187 [==============================] - 0s 250us/step - loss: 0.1499 - accuracy: 

10번 진행되기 때문에 accuracy 결과가 10개가 나왔습니다.
데이터셋 크기가 작기 때문에 accuracy가 높지는 않지만, 0.9, 0.85도 나왔는데, 이것은 아마 테스트셋이 잘 나눠지지 않아서 높게 나온 것으로 추측됩니다.